***
# Data Cleaning and Preparation
1. Clean data to - 
    i. Drop row level duplicates
    ii. Treat missing values
    iii. Change Data types to appropriate ones
2. evaluates central tendencies for the inspection score per year:
    i. For each type of vendor’s seating tyoe
    ii. For each ‘zip code’
    iii. seating capacity

3. Load all ststistics into database so as to referback easily.
***

In [1]:
#Import libraries
import pandas as pd
import toml
import os

In [2]:
#Set Pandas options
pd.set_option('display.max_columns', None)

In [3]:
#get current working directory
curr_dir = os.getcwd()
print(curr_dir)

C:\Users\guest1\MSc_Advanced_Programming\advanced-programming-assessment\data_processing


In [4]:
#import generic functions
from central_tendancies_function import central_tendancies
from data_processing_function import data_processing
from day0_data_preparation import prepare_data_day0

In [5]:
#Get directory for database operations to import create_df_from_sql function 
os.chdir('..\database_operations')

In [6]:
#import database functions
from create_df_from_sql_function import create_df_from_sql
from create_table_from_pandas_df_function import create_table_from_df

In [7]:
#change working directory to parent directory to import configs.
os.chdir('..')
wdir = os.getcwd()
print(wdir)

C:\Users\guest1\MSc_Advanced_Programming\advanced-programming-assessment


In [8]:
config = toml.load(wdir + "\config.toml")
print (config)

{'postgres_credentials': {'username': 'postgres', 'password': 'admin', 'host': '127.0.0.1', 'port': '5432'}, 'db_details': {'dbname': 'inspections_db', 'inspections_table': 'inspections', 'violations_table': 'violations', 'violations_agg_table': 'violations_aggregated', 'inspections_enriched_table': 'inspections_enriched', 'inspections_cleaned_table': 'inspections_cleaned', 'zip_statistics_table': 'statistics_zip_code', 'seating_type_statistics_table': 'statistics_seating_type', 'seating_capacity_statistics_table': 'statistics_seating_capacity', 'viz_violation_type': 'viz_violation_type', 'viz_num_violations': 'viz_num_violations', 'viz_geography': 'viz_geography', 'viz_zip_code': 'viz_zip_code'}}


In [11]:
#Using create_df_from_sql function create the pandas dataframe for inspections table.
#Pass connection config, dbname and table name to the function
inspections = create_df_from_sql(config['postgres_credentials'], config['db_details']['dbname'], config['db_details']['inspections_enriched_table'])

In [12]:
inspections.head(5)

,activity_date,owner_id,owner_name,facility_id,facility_name,record_id,program_name,program_status,program_element_pe,pe_desc,facility_address,facility_city,facility_state,facility_zip,service_code,service_desc,score,grade,serial_number,employee_id,location,supervisorial_district_boundaries,census_tracts_2010,board_approved_statistical_areas,zip_code,num_violations,violation_type
0,08/23/2018,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,22226 PALOS VERDES BLVD,TORRANCE,CA,90505,1,ROUTINE INSPECTION,97,A,DA2FXQNN6,EE0000126,POINT (-118.36927 33.826754),4,820,102,25719,3,"{47,46,33}"
1,12-06-17,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,22226 PALOS VERDES BLVD,TORRANCE,CA,90505,1,ROUTINE INSPECTION,95,A,DACP43IQW,EE0000126,POINT (-118.36927 33.826754),4,820,102,25719,5,"{47,44,40,35,34}"
2,06/23/2017,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,22226 PALOS VERDES BLVD,TORRANCE,CA,90505,1,ROUTINE INSPECTION,96,A,DAEMVMRBY,EE0000126,POINT (-118.36927 33.826754),4,820,102,25719,4,"{37,35,34,33}"
3,03/19/2019,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,22226 PALOS VERDES BLVD,TORRANCE,CA,90505,1,ROUTINE INSPECTION,96,A,DANER68S4,EE0000126,POINT (-118.36927 33.826754),4,820,102,25719,4,"{44,43,40,33}"
4,03-01-18,OW0000002,#1 CAFE INC,FA0056432,#1 CAFE,PR0045100,#1 CAFE,ACTIVE,1632,RESTAURANT (0-30) SEATS HIGH RISK,2080 CENTURY PARK E STE 108,LOS ANGELES,CA,90067,1,ROUTINE INSPECTION,90,A,DACZXQ74W,EE0000015,POINT (-118.412323 34.058815),3,660,265,24032,9,"{50,40,36,35,34,33,27,14,07}"


In [13]:
inspections.dtypes

activity_date                        object
owner_id                             object
owner_name                           object
facility_id                          object
facility_name                        object
record_id                            object
program_name                         object
program_status                       object
program_element_pe                    int64
pe_desc                              object
facility_address                     object
facility_city                        object
facility_state                       object
facility_zip                          int64
service_code                          int64
service_desc                         object
score                                 int64
grade                                object
serial_number                        object
employee_id                          object
location                             object
supervisorial_district_boundaries     int64
census_tracts_2010              

In [14]:
#Check for counts
inspections.count()

activity_date                        25
owner_id                             25
owner_name                           25
facility_id                          25
facility_name                        25
record_id                            25
program_name                         25
program_status                       25
program_element_pe                   25
pe_desc                              25
facility_address                     25
facility_city                        25
facility_state                       25
facility_zip                         25
service_code                         25
service_desc                         25
score                                25
grade                                25
serial_number                        25
employee_id                          25
location                             25
supervisorial_district_boundaries    25
census_tracts_2010                   25
board_approved_statistical_areas     25
zip_code                             25


In [15]:
#Check the unique values for program status
inspections['program_status'].value_counts()

ACTIVE      21
INACTIVE     4
Name: program_status, dtype: int64

In [16]:
# ***
# Call function data_processing to clean the data, including:
# 1. Drop row level duplicates
# 2. Treat missing values
# 3. Change Data types to appropriate ones
# ***

inspections_treated = data_processing(inspections)

In [17]:
#See sample records
inspections_treated.head(5)

,activity_date,owner_id,owner_name,facility_id,facility_name,record_id,program_name,program_status,program_element_pe,pe_desc,facility_address,facility_city,facility_state,facility_zip,service_code,service_desc,score,grade,serial_number,employee_id,location,supervisorial_district_boundaries,census_tracts_2010,board_approved_statistical_areas,zip_code,num_violations,violation_type,activity_yr,seating_capacity
0,2018-08-23,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT SEATS MODERATE RISK,22226 PALOS VERDES BLVD,TORRANCE,CA,90505,1,ROUTINE INSPECTION,97,A,DA2FXQNN6,EE0000126,POINT (-118.36927 33.826754),4,820,102,25719,3,"{47,46,33}",2018,(0-30)
1,2017-12-06,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT SEATS MODERATE RISK,22226 PALOS VERDES BLVD,TORRANCE,CA,90505,1,ROUTINE INSPECTION,95,A,DACP43IQW,EE0000126,POINT (-118.36927 33.826754),4,820,102,25719,5,"{47,44,40,35,34}",2017,(0-30)
2,2017-06-23,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT SEATS MODERATE RISK,22226 PALOS VERDES BLVD,TORRANCE,CA,90505,1,ROUTINE INSPECTION,96,A,DAEMVMRBY,EE0000126,POINT (-118.36927 33.826754),4,820,102,25719,4,"{37,35,34,33}",2017,(0-30)
3,2019-03-19,OW0000809,31,FA0019645,DREAM DINNERS,PR0045642,DREAM DINNERS,ACTIVE,1631,RESTAURANT SEATS MODERATE RISK,22226 PALOS VERDES BLVD,TORRANCE,CA,90505,1,ROUTINE INSPECTION,96,A,DANER68S4,EE0000126,POINT (-118.36927 33.826754),4,820,102,25719,4,"{44,43,40,33}",2019,(0-30)
4,2018-03-01,OW0000002,#1 CAFE INC,FA0056432,#1 CAFE,PR0045100,#1 CAFE,ACTIVE,1632,RESTAURANT SEATS HIGH RISK,2080 CENTURY PARK E STE 108,LOS ANGELES,CA,90067,1,ROUTINE INSPECTION,90,A,DACZXQ74W,EE0000015,POINT (-118.412323 34.058815),3,660,265,24032,9,"{50,40,36,35,34,33,27,14,07}",2018,(0-30)


In [18]:
#Check datatypes
inspections_treated.dtypes

activity_date                        datetime64[ns]
owner_id                                     object
owner_name                                   object
facility_id                                  object
facility_name                                object
record_id                                    object
program_name                                 object
program_status                               object
program_element_pe                            int64
pe_desc                                      object
facility_address                             object
facility_city                                object
facility_state                               object
facility_zip                                  int64
service_code                                  int64
service_desc                                 object
score                                         int64
grade                                        object
serial_number                                object
employee_id 

In [19]:
#Check for counts
inspections_treated.count()

activity_date                        21
owner_id                             21
owner_name                           21
facility_id                          21
facility_name                        21
record_id                            21
program_name                         21
program_status                       21
program_element_pe                   21
pe_desc                              21
facility_address                     21
facility_city                        21
facility_state                       21
facility_zip                         21
service_code                         21
service_desc                         21
score                                21
grade                                21
serial_number                        21
employee_id                          21
location                             21
supervisorial_district_boundaries    21
census_tracts_2010                   21
board_approved_statistical_areas     21
zip_code                             21


In [20]:
#Check the unique values for program status
inspections_treated['program_status'].value_counts()

ACTIVE    21
Name: program_status, dtype: int64

In [21]:
inspections_treated['seating_capacity'].value_counts()

(0-30)      11
(61-150)    10
Name: seating_capacity, dtype: int64

In [22]:
# ***
# Call function central_tendancies to get the mean, median, mode of score for the inspection score per year:
# 1. For each type of vendor’s seating
# 2. For each ‘zip code’
# ***
inputAtt = ['zip_code', 'activity_yr']
targetAtt='score'
df = inspections_treated

score_by_zip = central_tendancies(df, inputAtt, targetAtt)

In [23]:
score_by_zip.head(10)

,zip_code,activity_yr,median_score,mean_score,mode_score
0,23078,2016,95.0,95.0,95
1,23078,2017,93.0,93.0,93
2,23078,2018,94.0,94.0,90
3,24032,2017,90.0,90.0,90
4,24032,2018,91.0,91.0,90
5,24032,2019,90.0,90.0,90
6,24688,2016,95.0,95.0,95
7,24688,2017,95.0,95.0,95
8,24688,2018,95.0,95.0,93
9,24688,2019,90.0,90.0,90


In [24]:
inputAtt = ['pe_desc', 'activity_yr']
targetAtt='score'
df = inspections_treated

score_by_seating_type = central_tendancies(df, inputAtt, targetAtt)

In [25]:
score_by_seating_type.head(10)

,pe_desc,activity_yr,median_score,mean_score,mode_score
0,RESTAURANT SEATS HIGH RISK,2016,95.0,95.000000,95
1,RESTAURANT SEATS HIGH RISK,2017,93.0,92.600000,90
2,RESTAURANT SEATS HIGH RISK,2018,92.5,93.333333,90
3,RESTAURANT SEATS HIGH RISK,2019,90.0,90.000000,90
4,RESTAURANT SEATS MODERATE RISK,2017,95.5,95.500000,95
5,RESTAURANT SEATS MODERATE RISK,2018,97.0,97.000000,96
6,RESTAURANT SEATS MODERATE RISK,2019,96.0,96.000000,96


In [26]:
inputAtt = ['seating_capacity', 'activity_yr']
targetAtt='score'
df = inspections_treated

score_by_seating_capacity = central_tendancies(df, inputAtt, targetAtt)

In [27]:
score_by_seating_capacity.head(10)

,seating_capacity,activity_yr,median_score,mean_score,mode_score
0,(0-30),2017,92.5,92.750000,90
1,(0-30),2018,96.0,94.600000,90
2,(0-30),2019,93.0,93.000000,90
3,(61-150),2016,95.0,95.000000,95
4,(61-150),2017,95.0,94.333333,95
5,(61-150),2018,95.0,94.500000,90
6,(61-150),2019,90.0,90.000000,90


***
Test prepare_data_day0 function
***

In [9]:
os.chdir(curr_dir)

In [10]:
prepare_data_day0()

Loading inspections enriched data from database.
Inspections data load completed.
Treating inspections data to clean and process as per client's brief.
Data cleaning and processing completed.
Creating a table in Postgres DB and loading inspections cleaned data.
Table successfully created: inspections_cleaned
Data load for inspections cleaned data is completed.
Loading cleaned inspections data from database.
Inspections cleaned data load completed.
Calculating central tendencies for the inspection score per year for zip code
Loading in database
Table successfully created: statistics_zip_code
Data load for central tendencies data by zip is completed.
Loading in database
Table successfully created: statistics_seating_type
Data load for central tendencies data by type of vendor seating is completed.
Loading in database
Table successfully created: statistics_seating_capacity
Data load for central tendencies data by seating capacity is completed.
Loading in database - visualisation aggregati

C:\Users\guest1\MSc_Advanced_Programming\advanced-programming-assessment\data_processing\day0_data_preparation.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  type_of_vio_yr['violation_tp_list'] = type_of_vio_yr['violation_type'].apply(lambda x: x.replace('{', '').replace('}','').split(','))


Loading in database - visualisation aggregation by type of violations
Table successfully created: viz_violation_type
Data load for visualisation aggregation by type of violations is completed.
Loading in database - visualisation aggregation for geography
Table successfully created: viz_geography
Data load for visualisation aggregation for geography is completed.
Loading in database - visualisation aggregation by zip code
Table successfully created: viz_zip_code
Data load for visualisation aggregation by zip code is completed.
Process for data cleaning and preparation is completed.


True